EXTRACT DATA

In [196]:
client_id = ''      #ENTER CLIENT KEY
secret_key = ''     #ENTER SECRET KEY

In [197]:
import requests
import pandas as pd

In [198]:
auth = requests.auth.HTTPBasicAuth(client_id, secret_key)

In [199]:
with open('cred.txt', 'r') as f:   #CREATE cred.txt and write reddit password
    p = f.read()

In [200]:
data = {'grant_type': 'password',
        'username':'',             #Enter reddit username 
        'password':p}

In [201]:
headers = {'User-Agent':'Demo1/0.0.1'}

In [202]:
res = requests.post(f'https://www.reddit.com/api/v1/access_token',
                    auth = auth, data = data, headers=headers)

In [203]:
res

<Response [200]>

In [ ]:
res.json()

In [205]:
token = res.json()['access_token']
headers['Authorization'] = f'bearer {token}'

In [ ]:
headers

In [207]:
requests.get('https://oauth.reddit.com/api/v1/me', headers = headers)

<Response [200]>

In [208]:
link = 'https://oauth.reddit.com'

In [209]:
result  = requests.get(f'{link}/r/investing/new', headers = headers, params = {'limit' : 100} )

In [ ]:
result.json()

In [211]:
len(result.json()['data']['children'])

100

In [212]:
for i in result.json()['data']['children'][0]['data'].keys():
    print(i)

approved_at_utc
subreddit
selftext
author_fullname
saved
mod_reason_title
gilded
clicked
title
link_flair_richtext
subreddit_name_prefixed
hidden
pwls
link_flair_css_class
downs
top_awarded_type
hide_score
name
quarantine
link_flair_text_color
upvote_ratio
author_flair_background_color
subreddit_type
ups
total_awards_received
media_embed
author_flair_template_id
is_original_content
user_reports
secure_media
is_reddit_media_domain
is_meta
category
secure_media_embed
link_flair_text
can_mod_post
score
approved_by
is_created_from_ads_ui
author_premium
thumbnail
edited
author_flair_css_class
author_flair_richtext
gildings
content_categories
is_self
mod_note
created
link_flair_type
wls
removed_by_category
banned_by
author_flair_type
domain
allow_live_comments
selftext_html
likes
suggested_sort
banned_at_utc
view_count
archived
no_follow
is_crosspostable
pinned
over_18
all_awardings
awarders
media_only
can_gild
spoiler
locked
author_flair_text
treatment_tags
visited
removed_by
num_reports
di

In [213]:
# name, created_utc, subreddit, title, selftext, upvote_ratio, ups, downs, score

In [214]:
new_data = pd.DataFrame({
    'name':[], 
    'created_utc':[], 
    'subreddit':[], 
    'title':[], 
    'selftext':[], 
    'upvote_ratio':[], 
    'ups':[], 
    'downs':[], 
    'score':[]

})

In [215]:
new_data

,name,created_utc,subreddit,title,selftext,upvote_ratio,ups,downs,score


In [216]:
for i in result.json()['data']['children']:
    new_data = new_data.append({

    'name':i['data']['name'], 
    'created_utc':i['data']['created_utc'], 
    'subreddit':i['data']['subreddit'],
    'title':i['data']['title'],
    'selftext':i['data']['selftext'],
    'upvote_ratio':i['data']['upvote_ratio'],
    'ups':i['data']['ups'],
    'downs':i['data']['downs'],
    'score':i['data']['score']

    }, ignore_index = True)

In [217]:
new_data

,name,created_utc,subreddit,title,selftext,upvote_ratio,ups,downs,score
0,t3_14o83k9,1.688252e+09,investing,Can I invest with a credit card?,Recently in a good spot financially and was cu...,0.50,0.0,0.0,0.0
1,t3_14o7wcj,1.688251e+09,investing,Why gold is up still with Fed rate hike?,Gold went to $2000/oz from around March 2020 s...,1.00,1.0,0.0,1.0
2,t3_14o7jnv,1.688250e+09,investing,Ameritrade is changing to Schwab in Sept. What...,During the pandemic I bought some Ameritrade s...,0.50,0.0,0.0,0.0
3,t3_14o26le,1.688236e+09,investing,Best way to invest unused child support money ...,"Hello,\n\nMy kids’ father has been paying chil...",0.23,0.0,0.0,0.0
4,t3_14o1wzp,1.688236e+09,investing,"ELI5 Just starting life in USA, investing and ...",Some people advised me to refrain from investi...,0.25,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
95,t3_14jsf1k,1.687812e+09,investing,E-Trade Transfer to Morgan Stanley,"What is The Good, The Bad and/or The Ugly of t...",0.77,7.0,0.0,7.0
96,t3_14jr1si,1.687809e+09,investing,$CIO Opinion Piece - Discussion,8:29a ET 5/30/2023 - Dow Jones\nCity Office RE...,0.75,2.0,0.0,2.0
97,t3_14jo8gy,1.687802e+09,investing,How much saving Americans need to save for ret...,How much do U.S. adults think they need to sav...,0.90,502.0,0.0,502.0
98,t3_14jkuq1,1.687794e+09,investing,What is a good USD-Equivalent product to the C...,Does anyone know of a good equivalent to the [...,0.43,0.0,0.0,0.0


In [218]:
while True:

    result  = requests.get(f'{link}/r/investing/new', headers = headers, params = {'limit' : 1000, 'after': new_data['name'].iloc[len(new_data)-1]})
    print(result.json())
    print(len(result.json()['data']['children']))

    if len(result.json()['data']['children']) == 0:
        break

    for i in result.json()['data']['children']:
        new_data = new_data.append({

        'name':i['data']['name'], 
        'created_utc':i['data']['created_utc'], 
        'subreddit':i['data']['subreddit'],
        'title':i['data']['title'],
        'selftext':i['data']['selftext'],
        'upvote_ratio':i['data']['upvote_ratio'],
        'ups':i['data']['ups'],
        'downs':i['data']['downs'],
        'score':i['data']['score']

        }, ignore_index = True)

{'kind': 'Listing', 'data': {'after': 't3_14e5qwt', 'dist': 100, 'modhash': None, 'geo_filter': '', 'children': [{'kind': 't3', 'data': {'approved_at_utc': None, 'subreddit': 'investing', 'selftext': "I currently have about $1million invested In a stock account managed by a well-known financial advisor (national, advertise on TV, etc.).  They're pursuing an almost all equities strategy, which I am on board with and for the most part they've picked good stocks. \n\nMy issue is with the fees.  I pay them about $11-$12k a year (1%) which is fine except they are barely beating the market and sometimes lagging the market. They do provide me some other value in terms of information and planning.\n\nBut I'm just wondering: would I be better off keeping the $1M in diversified ETFs like SPY or QQQ, and firing the financial manager? \n\n (I have a much smaller fund that I keep in ETFs and it's doing just fine. Which has me wondering why I am bothering with the financial manager).\n\n\nEDIT, Than

In [219]:
new_data

,name,created_utc,subreddit,title,selftext,upvote_ratio,ups,downs,score
0,t3_14o83k9,1.688252e+09,investing,Can I invest with a credit card?,Recently in a good spot financially and was cu...,0.50,0.0,0.0,0.0
1,t3_14o7wcj,1.688251e+09,investing,Why gold is up still with Fed rate hike?,Gold went to $2000/oz from around March 2020 s...,1.00,1.0,0.0,1.0
2,t3_14o7jnv,1.688250e+09,investing,Ameritrade is changing to Schwab in Sept. What...,During the pandemic I bought some Ameritrade s...,0.50,0.0,0.0,0.0
3,t3_14o26le,1.688236e+09,investing,Best way to invest unused child support money ...,"Hello,\n\nMy kids’ father has been paying chil...",0.23,0.0,0.0,0.0
4,t3_14o1wzp,1.688236e+09,investing,"ELI5 Just starting life in USA, investing and ...",Some people advised me to refrain from investi...,0.25,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
856,t3_13i4klv,1.684147e+09,investing,Discussion on bank stocks...,The banking sector took a huge hit recently an...,0.87,180.0,0.0,180.0
857,t3_13i3cne,1.684143e+09,investing,Let us brew bucks with precision fermentation....,"Are you hungry too? I am, as always.\n\nI want...",0.31,0.0,0.0,0.0
858,t3_13i2oo8,1.684141e+09,investing,Daily General Discussion and Advice Thread - M...,Have a general question? Want to offer some c...,0.75,4.0,0.0,4.0
859,t3_13hypzb,1.684128e+09,investing,Advice for investing with Betterment,\nI have 75k I want to invest into betterment ...,0.67,3.0,0.0,3.0


In [221]:
new_data = new_data.replace({'|',''}, regex = True)

In [222]:
new_data.to_csv('reddit_data.csv', sep = '|', index = False)